<a href="https://colab.research.google.com/github/lolobq/ECGR-5105-Intro_To_Machine_Learning/blob/main/Homework6/Homework6Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, f1_score
import time

# Problem 2a

Create a fully connected Neural Network for all 10 classes in CIFAR-10 with only one hidden layer with a size of 512. Report your training time, training loss, and evaluation accuracy. Analyze your results in your report. Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [ ]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()  # Record the start time for training duration
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training time
    end_time = time.time()
    training_time = end_time - start_time

    # Print training statistics
    if epoch % 10 == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

    # Print total training time
    total_end_time = time.time()
    total_training_time = total_end_time - total_start_time
    print(f'Total Training Time: {total_training_time:.2f} seconds')

    # Testing the model
    model.eval()
    correct = 0
    total = 0
    all_predicted = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total

    # Print evaluation accuracy
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # Set the model back to training mode
    model.train()

# Calculate and print F1 score at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
print(f'Final F1 Score: {f1:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 2.0079088529662403, Training Time: 24.48 seconds
Total Training Time: 24.49 seconds
Test Accuracy: 34.54%
Total Training Time: 52.68 seconds
Test Accuracy: 37.04%
Total Training Time: 80.77 seconds
Test Accuracy: 38.15%
Total Training Time: 109.07 seconds
Test Accuracy: 38.78%
Total Training Time: 137.26 seconds
Test Accuracy: 39.33%
Total Training Time: 165.61 seconds
Test Accuracy: 39.70%
Total Training Time: 193.75 seconds
Test Accuracy: 40.21%
Total Training Time: 221.91 seconds
Test Accuracy: 40.45%
Total Training Time: 249.95 seconds
Test Accuracy: 40.75%
Total Training Time: 278.06 seconds
Test Accuracy: 41.24%
Epoch 11/300, Loss: 1.6962495709929015, Training Time: 24.35 seconds
Total Training Time: 305.94 seconds
Test Accuracy: 41.51%
Total Training Time: 333.71 seconds
Test Accuracy: 41.74%
Total Training Time: 361.72 seconds
Test Accuracy: 42.19

# Problem 2b

Extend your network with two more additional hidden layers, like the example we did in the lecture (pick the sizes of hidden layers properly). Train your network. Report your training time, loss, and evaluation accuracy after 300 epochs. Analyze your results in your report and compare your model size and accuracy over the baseline implementation in Problem 2.a. Do you see any over-fitting? Can you compare your model complexity against problem 2.a? Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, f1_score
import time

# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model with two additional hidden layers
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.ReLU(),
    nn.Linear(512, 256),  # Additional hidden layer 1
    nn.ReLU(),
    nn.Linear(256, 128),  # Additional hidden layer 2
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()  # Record the start time for training duration
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training time
    end_time = time.time()
    training_time = end_time - start_time

    # Print training statistics
    if epoch % 10 == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

    # Print total training time
    total_end_time = time.time()
    total_training_time = total_end_time - total_start_time
    print(f'Total Training Time: {total_training_time:.2f} seconds')

    # Testing the model
    model.eval()
    correct = 0
    total = 0
    all_predicted = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total

    # Print evaluation accuracy
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # Calculate and print F1 score
    f1 = f1_score(all_labels, all_predicted, average='weighted')
    print(f'F1 Score: {f1:.4f}')

    # Calculate and print confusion matrix
    cm = confusion_matrix(all_labels, all_predicted)
    print('Confusion Matrix:')
    print(cm)

    # Set the model back to training mode
    model.train()

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print final F1 score
final_f1 = f1_score(all_labels, all_predicted, average='weighted')
print(f'Final F1 Score: {final_f1:.4f}')

# Calculate and print final confusion matrix
final_cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(final_cm)



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 2.2967715345685136, Training Time: 25.31 seconds
Total Training Time: 25.31 seconds
Test Accuracy: 15.75%
F1 Score: 0.0786
Confusion Matrix:
[[  0   0   0   0   6   0  72   0 273 649]
 [  0   0   0   0  19   0 211   0  93 677]
 [  0   0   0   0  12   0 273   0 101 614]
 [  0   0   0   0  19   0 329   0  50 602]
 [  0   0   0   0   9   0 437   0  28 526]
 [  0   0   0   0  28   0 288   0  65 619]
 [  0   0   0   0  14   0 437   0  17 532]
 [  0   0   0   0  16   0 272   0  57 655]
 [  0   0   0   0  14   0  76   0 302 608]
 [  0   0   0   0   7   0  87   0  79 827]]
Total Training Time: 54.18 seconds
Test Accuracy: 19.07%
F1 Score: 0.1013
Confusion Matrix:
[[  0   2   1   0   8   0  97   0 529 363]
 [  0  29   0   0   7   1 270   0 202 491]
 [  0   4   0   0  12   0 395   0 186 403]
 [  0   4   0   1  15   3 455   0  93 429]
 [  0   1   0   0  12   0 564  